## Panggil dataset
Sekarang, kita akan mencoba mengaplikasikan transfer learning untuk aplikasi klasifikasi 3 kelas kasus wine.csv

In [74]:
# Panggil library yang digunakan
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Panggil dataset
dataset = pd.read_csv('Wine.csv')
# Tunjukkan 5 data pertama dataset
dataset.head()

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1


In [85]:
dataset = dataset[~(dataset['Customer_Segment'] == 3)]
dataset

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline,Customer_Segment
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,12.07,2.16,2.17,21.0,85,2.60,2.65,0.37,1.35,2.76,0.86,3.28,378,2
126,12.43,1.53,2.29,21.5,86,2.74,3.15,0.39,1.77,3.94,0.69,2.84,352,2
127,11.79,2.13,2.78,28.5,92,2.13,2.24,0.58,1.76,3.00,0.97,2.44,466,2
128,12.37,1.63,2.30,24.5,88,2.22,2.45,0.40,1.90,2.12,0.89,2.78,342,2


In [86]:
X = dataset.iloc[:, 0:13].values
y = dataset.iloc[:, 13].values
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [87]:
# Applying PCA
from sklearn.decomposition import PCA
# selain memasukkan jumlah, bisa juga dimasukkan jumlah persen cummulative sum saja, nilainya antara 0 sampai 1
pca = PCA(n_components = 2)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

## Panggil model
Panggil model 'best_model.h5' yang sudah kita buat pada materi 'Early Stopping'

In [163]:
from keras import models
from keras import layers
model_A = models.load_model('best_model.h5')
model_A.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 500)               1500      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 501       
Total params: 2,001
Trainable params: 2,001
Non-trainable params: 0
_________________________________________________________________


Buang layer paling akhir dan akan kita ganti dengan output berjumlah 3 (karena ada 3 kelas)

In [164]:
# Definisikan sequential berdasarkan model_A yang dibuang layer paling akhirnya
model_B_on_A = models.Sequential(model_A.layers[:-1])
model_B_on_A.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 500)               1500      
Total params: 1,500
Trainable params: 1,500
Non-trainable params: 0
_________________________________________________________________


Tambahkan output layer yang baru

In [165]:
model_B_on_A.add(layers.Dense(1, activation='tanh'))
model_B_on_A.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 500)               1500      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 501       
Total params: 2,001
Trainable params: 2,001
Non-trainable params: 0
_________________________________________________________________


In [166]:
model_A_clone = models.clone_model(model_A)
# Pindahkan nilai bobot weights
model_A_clone.set_weights(model_A.get_weights())

Kunci layer paling awal supaya tidak diganti nilai bobotnya pada saat proses pelatihan

In [167]:
for layer in model_B_on_A.layers[:-1]:
    layers.trainable = True

Compile model

In [168]:
model_B_on_A.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

Latih model

In [169]:
history = model_B_on_A.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
4/4 [==============================] - 1s 76ms/step - loss: 21.6288 - accuracy: 0.1018 - val_loss: -6.4516 - val_accuracy: 0.5769
Epoch 2/5
4/4 [==============================] - 0s 14ms/step - loss: -8.6657 - accuracy: 0.4317 - val_loss: -6.4516 - val_accuracy: 0.5769
Epoch 3/5
4/4 [==============================] - 0s 13ms/step - loss: -9.3646 - accuracy: 0.3859 - val_loss: -6.4516 - val_accuracy: 0.5769
Epoch 4/5
4/4 [==============================] - 0s 13ms/step - loss: -8.7292 - accuracy: 0.4276 - val_loss: -6.4516 - val_accuracy: 0.5769
Epoch 5/5
4/4 [==============================] - 0s 13ms/step - loss: -8.8881 - accuracy: 0.4171 - val_loss: -6.4516 - val_accuracy: 0.5769


Hasil yang diperoleh kurang baik, hal ini disebabkan penggunaan transfer learning sebenarnya hanya baik untuk kasus deep neural network yang memiliki banyak layer. Kita akan detailkan kembali di dalam mater minggu depan di dalam aplikias deep learning untuk computer vision.